# Trabalho em Grupo I

```
Disciplina: Inteligência artificial 6º semestre
IFSP campus Campinas
Alunos: 
- Paulo Henrique Belucci - CP3001369
- Silvio da Col de Brito - CP300130X
Prof.: Dr. Samuel Martins
```

## Análise exploratória - Pandemia do Corona vírus no Brasil

---

### Descrição e motivação

A pandemia do Corona vírus teve início ao final do ano 2019 e, em pouco mais de um ano, é responsável por mais de 1,4 milhão de mortes em todo o mundo. Seus sintomas são parecidos com os sintomas da gripe, com destaque para a síndrome respiratória. Embora sua taxa de letalidade não seja tão alta, seu sintomas somados a um grande número de infectados tem se tornado um gravíssimo problema de saúde pública em todos os países, devido a sobrecarga de leitos de Unidade de Tratamento Intensivo (UTI) por aqueles casos se se agravam.

Muitos estudos e pesquisas tem sido desenvolvidos em prol de mapear a genética do vírus, entender as formas de contágio, testar tratamentos para pacientes infectados e, destaca-se, desenvolver uma vacina para imunizar a população interrompendo assim o ciclo de contágio.

Esta análise exploratória visa entender alguns aspectos da pandemia no Brasil, tais informações servem de insumo para elaboração de estratégias eficazes no combate ao avança da doença.



### Descrição da base de dados

O conjunto de dados foi extraído do [painel oficial](https://covid.saude.gov.br/) de COVID-19 no Brasil e cobre informações de todos os estados e municípios.

Cada registro contabiliza os casos novos, acumulados, recuperados por estado e município. A periodicidade é diária, ou seja, cada linha representa um dia.

As colunas presentes no _dataset_ são:
- regiao
- estado
- municipio
- coduf
- codmun
- codRegiaoSaude
- nomeRegiaoSaude
- data
- semanaEpi
- populacaoTCU2019
- casosAcumulado
- casosNovos
- obitosAcumulado
- obitosNovos
- Recuperadosnovos
- emAcompanhamentoNovos
- interior/metropolitana

Na sequência é realizada uma análise prévia sobre o dataset, seus tipos de dados, numero de registros, dados faltantes:

In [1]:
import numpy as np
import pandas as pd

In [2]:
covid = pd.read_csv('HIST_PAINEL_COVIDBR_02jan2021.csv', delimiter=';')
covid.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN


In [3]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18187 entries, 0 to 18186
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   regiao                  18187 non-null  object 
 1   estado                  17873 non-null  object 
 2   municipio               9140 non-null   object 
 3   coduf                   18187 non-null  int64  
 4   codmun                  9422 non-null   float64
 5   codRegiaoSaude          9140 non-null   float64
 6   nomeRegiaoSaude         9140 non-null   object 
 7   data                    18187 non-null  object 
 8   semanaEpi               18187 non-null  int64  
 9   populacaoTCU2019        17904 non-null  float64
 10  casosAcumulado          18186 non-null  float64
 11  casosNovos              18186 non-null  float64
 12  obitosAcumulado         18186 non-null  float64
 13  obitosNovos             18186 non-null  float64
 14  Recuperadosnovos        260 non-null  

### Preparação da base de dados

Infelizmente as colunas de estados e municiṕios possuem muitos dados faltantes. No caso dos estados, alguns podem ser recuperados com base no respectivo código 'coduf'. Já no caso de municípios, os códigos também possuem muitos campos faltantes.

In [4]:
# Gerando um dicionário com os códigos dos estados
cod_estados = covid[['coduf', 'estado']].groupby('estado').mean().sort_values('coduf')
cod_estados.reset_index(inplace=True)
cod_estados = cod_estados.set_index('coduf').to_dict()['estado']
cod_estados[np.NaN]  = np.NaN
cod_estados

{11: 'RO',
 12: 'AC',
 13: 'AM',
 14: 'RR',
 15: 'PA',
 16: 'AP',
 17: 'TO',
 21: 'MA',
 22: 'PI',
 23: 'CE',
 24: 'RN',
 25: 'PB',
 26: 'PE',
 27: 'AL',
 28: 'SE',
 29: 'BA',
 31: 'MG',
 32: 'ES',
 33: 'RJ',
 35: 'SP',
 41: 'PR',
 42: 'SC',
 43: 'RS',
 50: 'MS',
 51: 'MT',
 52: 'GO',
 53: 'DF',
 nan: nan}

In [5]:
# Codigo inválido: 76
print(' ======  Códigos de UF  ======')
print(covid.coduf.unique(), '\n')
print(' ======  Código inválido  ======')
print(f'Código 76  => quantidade: {len(covid[covid.coduf == 76])}\n\n')

# Excluindo registros com código inválido
covid = covid[covid.coduf != 76]
covid.head(5)

 ======  Códigos de UF  ======
[76 11 12 13 14 15 16 17 21 22 23 24 25 26 27 28 29 31 32 33 35 41 42 43
 50 51 52 53] 

 ======  Código inválido  ======
Código 76  => quantidade: 314




,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
314,Norte,RO,NaN,11,NaN,NaN,NaN,2020-02-25,9,1777225.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
315,Norte,RO,NaN,11,NaN,NaN,NaN,2020-02-26,9,1777225.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
316,Norte,RO,NaN,11,NaN,NaN,NaN,2020-02-27,9,1777225.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
317,Norte,RO,NaN,11,NaN,NaN,NaN,2020-02-28,9,1777225.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
318,Norte,RO,NaN,11,NaN,NaN,NaN,2020-02-29,9,1777225.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


In [6]:
# Preenchendo os estados faltantes com seus respectivos códigos
estados = [cod_estados[cod] for cod in covid.coduf]
print(f'Estados preenchidos: {len(estados)}\n')
covid.estado = estados

## Convertendo as datas do tipo 'object' para o tipo 'datetime'
covid.data = covid.data.apply(lambda dt : pd.to_datetime(dt))

covid.info()

Estados preenchidos: 17873

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17873 entries, 314 to 18186
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   regiao                  17873 non-null  object        
 1   estado                  17873 non-null  object        
 2   municipio               9140 non-null   object        
 3   coduf                   17873 non-null  int64         
 4   codmun                  9422 non-null   float64       
 5   codRegiaoSaude          9140 non-null   float64       
 6   nomeRegiaoSaude         9140 non-null   object        
 7   data                    17873 non-null  datetime64[ns]
 8   semanaEpi               17873 non-null  int64         
 9   populacaoTCU2019        17590 non-null  float64       
 10  casosAcumulado          17872 non-null  float64       
 11  casosNovos              17872 non-null  float64       
 12  obitosAcumulado 

In [12]:
# Montando dataset sem as colunas relacionadas a municipio
covid_estados = covid[['regiao', 
                       'estado', 
                       'data', 
                       'semanaEpi', 
                       'populacaoTCU2019',
                       'casosAcumulado', 
                       'casosNovos', 
                       'obitosAcumulado', 
                       'obitosNovos']]

# Excluindo os poucos registros nulo restantes
covid_estados = covid_estados.dropna()

# Reinicialiando o índice
covid_estados.reset_index(inplace=True)
covid_estados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17590 entries, 0 to 17589
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   index             17590 non-null  int64         
 1   regiao            17590 non-null  object        
 2   estado            17590 non-null  object        
 3   data              17590 non-null  datetime64[ns]
 4   semanaEpi         17590 non-null  int64         
 5   populacaoTCU2019  17590 non-null  float64       
 6   casosAcumulado    17590 non-null  float64       
 7   casosNovos        17590 non-null  float64       
 8   obitosAcumulado   17590 non-null  float64       
 9   obitosNovos       17590 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(2), object(2)
memory usage: 1.3+ MB


### Análise Exploratória

Inicialmente, não considerando as colunas relacionadas a municípios - por possuirem muitos dados faltantes - algumas análises poderão ser feitas com o _dataset_.  

---
1. Qual o número de mortes no país até o registro mais recente?
2. Qual o número de infectados no país até o registro mais recente?
3. Qual o número de mortes em cada estado até o registro mais recente?
4. Qual o número de infectados em cada estado até o registro mais recente?
---
5. Qual a taxa de letalidade no país?
7. Qual a taxa de letalidade por estado?
---
8. Como foi a evolução de novos casos vs mortes no país?
9. Como foi a evolução de novos casos vs mortes para cada estado?
---


In [29]:
covid_por_estado = covid_estados.groupby('estado')
covid_por_estado.groups.keys()

dict_keys(['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO'])

In [30]:
covid_por_estado.groups['AC']

Int64Index([313, 314, 315, 316, 317, 318, 319, 320, 321, 322,
            ...
            616, 617, 618, 619, 620, 621, 622, 623, 624, 625],
           dtype='int64', length=313)

In [31]:
covid_estados.iloc[1000:1020, :]

,index,regiao,estado,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos
1000,1314,Norte,RR,2020-04-26,18,605761.0,401.0,56.0,4.0,1.0
1001,1315,Norte,RR,2020-04-27,18,605761.0,407.0,6.0,4.0,0.0
1002,1316,Norte,RR,2020-04-28,18,605761.0,425.0,18.0,6.0,2.0
1003,1317,Norte,RR,2020-04-29,18,605761.0,452.0,27.0,6.0,0.0
1004,1318,Norte,RR,2020-04-30,18,605761.0,519.0,67.0,7.0,1.0
1005,1319,Norte,RR,2020-05-01,18,605761.0,602.0,83.0,8.0,1.0
1006,1320,Norte,RR,2020-05-02,18,605761.0,668.0,66.0,9.0,1.0
1007,1321,Norte,RR,2020-05-03,19,605761.0,740.0,72.0,11.0,2.0
1008,1322,Norte,RR,2020-05-04,19,605761.0,806.0,66.0,11.0,0.0
1009,1323,Norte,RR,2020-05-05,19,605761.0,869.0,63.0,11.0,0.0
